# Xanadu Quantum Computing Platforms: Real-World Applications

This notebook demonstrates practical quantum computing applications using Xanadu's platforms:
- **PennyLane**: Quantum machine learning and hybrid quantum-classical algorithms
- **Strawberry Fields**: Photonic quantum computing and continuous variable systems

## Installation
```bash
pip install pennylane strawberryfields pennylane-sf matplotlib numpy scikit-learn
```

In [ ]:
# Import required libraries
import pennylane as qml
import strawberryfields as sf
from strawberryfields.ops import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')

print(f"PennyLane version: {qml.__version__}")
print(f"Strawberry Fields version: {sf.__version__}")

## Part 1: Quantum Machine Learning with PennyLane

### Example 1: Variational Quantum Classifier
A practical quantum machine learning example for binary classification.

In [ ]:
# Create a quantum device
n_qubits = 4
dev = qml.device('default.qubit', wires=n_qubits)

# Define the quantum circuit
@qml.qnode(dev)
def quantum_classifier(x, params):
    # Data encoding
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))
    
    # Variational layers
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

# Generate sample data
X, y = make_classification(n_samples=100, n_features=4, n_classes=2, 
                          n_redundant=0, random_state=42)
X = normalize(X) * np.pi  # Normalize for angle encoding
y = 2 * y - 1  # Convert to {-1, 1}

# Initialize parameters
n_layers = 3
params = np.random.normal(0, np.pi, (n_layers, n_qubits, 3))

print(f"Data shape: {X.shape}")
print(f"Parameter shape: {params.shape}")
print(f"Sample prediction: {quantum_classifier(X[0], params)}")

In [ ]:
# Training the quantum classifier
def cost_function(params, X, y):
    predictions = [quantum_classifier(x, params) for x in X]
    return np.mean((predictions - y) ** 2)

# Optimize using PennyLane's built-in optimizers
optimizer = qml.AdamOptimizer(stepsize=0.1)
costs = []

for epoch in range(50):
    params, cost = optimizer.step_and_cost(
        lambda p: cost_function(p, X[:20], y[:20]), params
    )
    costs.append(cost)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Cost = {cost:.4f}")

# Plot training progress
plt.figure(figsize=(10, 5))
plt.plot(costs)
plt.title('Quantum Classifier Training Progress')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.grid(True)
plt.show()

### Example 2: Quantum Feature Map
Demonstrating quantum feature mapping for enhanced classical ML.

In [ ]:
# Quantum feature map
@qml.qnode(dev)
def quantum_feature_map(x, params):
    # Encode classical data
    for i in range(len(x)):
        qml.RY(x[i], wires=i)
    
    # Apply quantum gates with parameters
    for i in range(n_qubits):
        qml.RZ(params[i], wires=i)
    
    # Entangling gates
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    
    # Return quantum features
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Generate quantum features
feature_params = np.random.normal(0, np.pi, n_qubits)
quantum_features = [quantum_feature_map(x[:4], feature_params) for x in X[:10]]

print("Original features shape:", X[:10].shape)
print("Quantum features shape:", np.array(quantum_features).shape)
print("\nSample quantum features:")
print(np.array(quantum_features)[:3])

## Part 2: Photonic Quantum Computing with Strawberry Fields

### Example 3: Gaussian Boson Sampling
A key application in photonic quantum computing for demonstrating quantum advantage.

In [ ]:
# Gaussian Boson Sampling circuit
def gaussian_boson_sampling(n_modes=4, n_layers=2):
    prog = sf.Program(n_modes)
    
    with prog.context as q:
        # Initial squeezed states
        for i in range(n_modes):
            Sgate(0.5) | q[i]  # Squeezing parameter
        
        # Interferometer layers
        for layer in range(n_layers):
            for i in range(0, n_modes - 1, 2):
                BSgate(np.pi/4, 0) | (q[i], q[i + 1])  # Beam splitter
            
            for i in range(n_modes):
                Rgate(np.random.uniform(0, 2*np.pi)) | q[i]  # Rotation
            
            for i in range(1, n_modes - 1, 2):
                BSgate(np.pi/4, 0) | (q[i], q[i + 1])
    
    return prog

# Create and run the GBS circuit
gbs_prog = gaussian_boson_sampling()

# Use the Gaussian backend for simulation
eng = sf.Engine('gaussian')
result = eng.run(gbs_prog)

print("Gaussian Boson Sampling Program:")
print(gbs_prog)
print(f"\nFinal state means: {result.state.means()}")
print(f"Covariance matrix shape: {result.state.cov().shape}")

In [ ]:
# Visualize the covariance matrix
cov_matrix = result.state.cov()

plt.figure(figsize=(8, 6))
plt.imshow(cov_matrix, cmap='viridis')
plt.colorbar(label='Covariance')
plt.title('Covariance Matrix from Gaussian Boson Sampling')
plt.xlabel('Mode Index')
plt.ylabel('Mode Index')
plt.show()

### Example 4: Quantum Machine Learning with Continuous Variables
Using photonic quantum computing for ML tasks.

In [ ]:
# CV Quantum Neural Network
def cv_qnn(x, params, n_modes=2):
    prog = sf.Program(n_modes)
    
    with prog.context as q:
        # Data encoding
        for i in range(min(len(x), n_modes)):
            Dgate(x[i], 0) | q[i]  # Displacement encoding
        
        # Variational layer
        for i in range(n_modes):
            Sgate(params[i]) | q[i]  # Squeezing
            Rgate(params[i + n_modes]) | q[i]  # Rotation
        
        # Entangling layer
        BSgate(params[-2], params[-1]) | (q[0], q[1])
    
    return prog

# Generate sample data for CV-QNN
cv_data = np.random.normal(0, 0.5, (10, 2))
cv_params = np.random.normal(0, 0.1, 6)  # 2*n_modes + 2 parameters

# Run CV-QNN
cv_prog = cv_qnn(cv_data[0], cv_params)
cv_result = eng.run(cv_prog)

print("CV Quantum Neural Network:")
print(f"Input data: {cv_data[0]}")
print(f"Parameters: {cv_params}")
print(f"Output means: {cv_result.state.means()}")

## Part 3: Hybrid Quantum-Classical Applications

### Example 5: Quantum Optimization (QAOA)
Solving combinatorial optimization problems using quantum algorithms.

In [ ]:
# QAOA for Max-Cut problem
def qaoa_maxcut(graph_edges, n_qubits, p_layers=2):
    dev_qaoa = qml.device('default.qubit', wires=n_qubits)
    
    @qml.qnode(dev_qaoa)
    def qaoa_circuit(params):
        # Initial superposition
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
        
        # QAOA layers
        for p in range(p_layers):
            # Cost Hamiltonian
            for edge in graph_edges:
                qml.CNOT(wires=[edge[0], edge[1]])
                qml.RZ(params[p], wires=edge[1])
                qml.CNOT(wires=[edge[0], edge[1]])
            
            # Mixer Hamiltonian
            for i in range(n_qubits):
                qml.RX(params[p + p_layers], wires=i)
        
        # Expectation value of cost function
        cost = 0
        for edge in graph_edges:
            cost += 0.5 * (1 - qml.expval(qml.PauliZ(edge[0]) @ qml.PauliZ(edge[1])))
        
        return cost
    
    return qaoa_circuit

# Define a simple graph (triangle)
edges = [(0, 1), (1, 2), (2, 0)]
n_nodes = 3
qaoa_func = qaoa_maxcut(edges, n_nodes)

# Optimize QAOA parameters
qaoa_params = np.random.uniform(0, 2*np.pi, 4)  # 2 * p_layers
qaoa_optimizer = qml.AdagradOptimizer(stepsize=0.5)

print("QAOA Max-Cut Optimization:")
for i in range(20):
    qaoa_params, cost = qaoa_optimizer.step_and_cost(
        lambda p: -qaoa_func(p), qaoa_params  # Maximize (minimize negative)
    )
    if i % 5 == 0:
        print(f"Step {i}: Cost = {-cost:.4f}")

print(f"\nOptimal parameters: {qaoa_params}")
print(f"Maximum cut value: {-qaoa_func(qaoa_params):.4f}")

## Part 4: Real-World Application: Quantum Chemistry

### Example 6: Variational Quantum Eigensolver (VQE)
Finding ground state energies of molecules.

In [ ]:
# VQE for H2 molecule
def h2_hamiltonian():
    """Simplified H2 Hamiltonian in minimal basis"""
    coeffs = [-1.0523732, 0.39793742, -0.39793742, -0.01128010, 0.18093119]
    obs = [
        qml.Identity(0),
        qml.PauliZ(0),
        qml.PauliZ(1),
        qml.PauliZ(0) @ qml.PauliZ(1),
        qml.PauliX(0) @ qml.PauliX(1)
    ]
    return qml.Hamiltonian(coeffs, obs)

# VQE circuit
dev_vqe = qml.device('default.qubit', wires=2)
hamiltonian = h2_hamiltonian()

@qml.qnode(dev_vqe)
def vqe_circuit(params):
    # Prepare initial state (HF state for H2)
    qml.PauliX(wires=0)
    
    # Ansatz: UCCSD-inspired circuit
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=1)
    
    return qml.expval(hamiltonian)

# Optimize VQE
vqe_params = np.random.normal(0, 0.1, 3)
vqe_optimizer = qml.AdamOptimizer(stepsize=0.1)

print("VQE for H2 molecule:")
energies = []
for i in range(50):
    vqe_params, energy = vqe_optimizer.step_and_cost(vqe_circuit, vqe_params)
    energies.append(energy)
    if i % 10 == 0:
        print(f"Step {i}: Energy = {energy:.6f} Hartree")

# Plot convergence
plt.figure(figsize=(10, 5))
plt.plot(energies)
plt.axhline(y=-1.136189, color='r', linestyle='--', 
           label='Exact ground state')
plt.title('VQE Convergence for H₂ Molecule')
plt.xlabel('Iteration')
plt.ylabel('Energy (Hartree)')
plt.legend()
plt.grid(True)
plt.show()

print(f"\nFinal ground state energy: {energies[-1]:.6f} Hartree")
print(f"Exact ground state energy: -1.136189 Hartree")
print(f"Error: {abs(energies[-1] + 1.136189):.6f} Hartree")

## Summary and Next Steps

This notebook demonstrated several real-world applications of Xanadu's quantum computing platforms:

### PennyLane Applications:
1. **Quantum Machine Learning**: Variational quantum classifiers for supervised learning
2. **Quantum Feature Maps**: Enhanced feature representation for classical ML
3. **Quantum Optimization**: QAOA for combinatorial problems
4. **Quantum Chemistry**: VQE for molecular ground state calculations

### Strawberry Fields Applications:
1. **Gaussian Boson Sampling**: Demonstrating quantum advantage in sampling
2. **Continuous Variable QML**: Photonic quantum neural networks

### Key Advantages:
- **Hardware Agnostic**: Code runs on simulators and real quantum hardware
- **Differentiable Programming**: Seamless integration with classical ML frameworks
- **Photonic Focus**: Specialized tools for continuous variable quantum computing
- **Production Ready**: Enterprise-grade tools for quantum application development

### Next Steps:
1. **Scale Up**: Try larger problems with more qubits/modes
2. **Real Hardware**: Connect to Xanadu's cloud quantum computers
3. **Hybrid Algorithms**: Combine quantum and classical processing
4. **Domain Applications**: Apply to specific use cases (finance, chemistry, ML)

For more examples and documentation:
- [PennyLane Documentation](https://pennylane.ai/)
- [Strawberry Fields Documentation](https://strawberryfields.ai/)
- [Xanadu Quantum Cloud](https://cloud.xanadu.ai/)